# PVseek tutorial 

In this tutorial, we will show how to use PVseek to diagnose plant viruses using different highthroughput sequencing data, such as Illumina single/paired-end reads, small RNA reads, Oxford Nanopore reads. Some datasets can be found in PVseek/test/data. You can choose the interesting part and skip the other.
1. [Illumina paried-end reads](#pair)
2. [Illumina single reads](#single)
3. [Oxford Nanopore reads](#nanopore)
4. [Illumina small RNA reads](#small)
5. [HiPlex/Amplicon reads](#hiplex)

## <a id='pair'></a>1. Illumina paired-end reads

We use the data [Dataset_8](https://gitlab.com/ilvo/VIROMOCKchallenge/-/blob/master/Datasets/Dataset8.md) from [VIROMOCK challenge](https://gitlab.com/ilvo/VIROMOCKchallenge) datasets from the European Plant Health Bioinformatics Network (PHBN).

### Step 1. set up config.yaml
The config file for paired-end reads is config_pe.yaml in the PVseek folder. We can directly use it after changing database file paths.

Here are some explanation for config_pe.yaml.

Our data file names are Dataset_8_R1.fastq.gz and Dataset_8_R2.fastq.gz. In the config_ye.yaml file

    seq_type: 'pe' #paired-end
    strand1: 'R1'  
    strand2: 'R2'
    input_format: 'fastq.gz'

Fastp is used to trim reads. We use a sliding window (size 4) to trim reads with mean quality 20 and minimum length 30.  

    fastp_pe_param: "--length_required 30 --cut_right --cut_window_size 4 --cut_mean_quality 20" 

Bowtie2 is used to map reads to reference genomes. Anonymous reads can be mapped to 100 positons (-k 100)

    mappingTool: bowtie2 #bwa 
    bowtie2_param: "--very-sensitive-local -k 100 --score-min L,20,1.0" 

PVseek uses two thresholds to filter mapping results: minimum read mapped to a virus (readNumThd) and minimum genome coverage for a virus/fragment (genomeCovThd). You can setup them according to your positive controls and negative controls.

    readNumThd: 100 #minimu read number (100) for a virus to be selected
    genomeCovThd: 0.20 #minimum genome coverage (20%) for a virus to be selected

The virus coverage graph can be plotted if it's enabled by "yes".
    
    coverage: "no"  #"yes"   

The database file paths are needed to be changed to your local PVseek folder.

    virusDb: /path/to/PVseek/db/plantvirus_genome.fasta 
    virusTaxon: /path/to/PVseek/db/plantvirus.gb_taxon.txt 
    viralRefInfo: /path/to/PVseek/db/plantvirus.info.txt 

If you know some sequence annotations are questionalbe, please add their accesions in unwantedSeqId.txt

    badSeqIds: /paht/to/PVseek/db/unwantedSeqId.txt 

Please keep the other key/values in config_pe.yaml, do not change them.

### Step 2. run PVseek

Suppose Dataset_8 two read files are in PVseek test folder: /path/to/PVseek/test/data, the work folder is ~/test_PVseek_pe, the PVseek program is in /path/to/PVseek, 16 CPU cores are used. Then we can run a dry-test first

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_pe.yaml -s /path/to/PVseek/Snakefile --config workDir=~/test_PVseek_pe --fastqDir=/path/to/PVseek/test/data --core 16 -n

If dry-test is ok, we can run PVseek without "-n"

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_pe.yaml -s /path/to/PVseek/Snakefile --config workDir=~/test_PVseek_pe --fastqDir=/path/to/PVseek/test/data --core 16

The job should be done in a few minutes if the database were indexed before.

### Step 3. check results 

The filtered results are in the file ~/test_PVseek_pe/report/report.txt.
The unfiltered results are in the file ~/test_PVseek_pe/report/report.unfiltered.txt.
The read quality and numbers before and after trimming are in the file ~/test_PVseek_pe/report/qcReadNumber.txt.

**Pelargonium flower break virus (PFBV) and Chenopodium quinoa mitovirus 1 (CqMV1) are detected.**

**Here is the report table** 

| Sample | TaxonId | ReadInTaxon | Reference | RefCoverage | Virus | TaxonPath | RefDescription |  
| :---: | :---: | :---: | :---: | :---: | :--- | :--- | :--- |  
|Dataset_8|35291|53630|AJ514833.1|1.0|Pelargonium flower break virus|Viruses; Riboviria; Orthornavirae; Kitrinoviricota; Tolucaviricetes; Tolivirales; Tombusviridae; Procedovirinae; Alphacarmovirus; Pelargonium flower break virus;|Pelargonium flower break carmovirus complete genome| 
|Dataset_8|2185087|288|NC_040543.1|0.97|Chenopodium quinoa mitovirus 1|Viru ses; Riboviria; Orthornavirae; Lenarviricota; Howeltoviricetes; Cryppavirales; Mitoviridae; Duamitovirus; Duamitovirus chqu1; Chenopodium quinoa mitovirus 1;|Chenopodium quinoa mitovirus 1 isolate Che1, complete genome >MF375475.1 Chenopodium quinoa mitovirus 1 isolate Che1, complete genome| 


**Here is the structure of the working directory (~/test_PVseek_pe)**

In [ ]:
!cd ~/test_PVseek_pe
!tree

```
Dataset_8
├── logs
│   ├── checkPoint
│   │   └── Dataset_8.mapping.done
│   ├── fastp
│   │   └── Dataset_8.log
│   └── mapping
│       └── Dataset_8.map2Ref.log
├── mapping
│   ├── Dataset_8.bam
│   ├── Dataset_8.bam.bai
│   ├── Dataset_8.coverage.txt
│   ├── Dataset_8.mappedRead.txt
│   ├── Dataset_8.mappedRef.filtered.txt
│   └── Dataset_8.mappedRef.full.txt
├── raw
│   ├── Dataset_8_R1_001.fastq.gz
│   └── Dataset_8_R2_001.fastq.gz
├── report
│   ├── qcReadNumber.txt
│   ├── report.txt
│   └── report.unfiltered.txt
└── trimmed
    ├── Dataset_8.fastp.html
    ├── Dataset_8.fastp.json
    ├── Dataset_8_R1.trimmed.fastq.gz
    └── Dataset_8_R2.trimmed.fastq.gz
```

**Here is explanation for folders:**
1. Read file folders:
    - "raw" folder contains all raw fastq files
    - "trimmed" folder contains fastq files after trimming low quality reads and fastp trimming reports (*.fastp.html and *.fastp.json)
2. Mapping reads to references folder:
    - "mapping" folder contains mapped bam file, unfiltered mapping result(*.mappedRef.full.txt) and filtered mapping result (*.mappedRef.filtered.txt) 
    - "coverage" folder contains viral genome coverage graphs and consensus sequences if the key "coverage" is enabled ("yes") in the config file
3. Report folder:
    - "report" folder contains filtered reports (report.txt), unfiltered report (report.unfiltered.txt) and read quality table (qcReadNumber.txt)
4. Log folder:
    - "logs" folder contains log files from different tools


## Viral genome sequences are missed in the database

Let's test [Dataset_9](https://gitlab.com/ilvo/VIROMOCKchallenge/-/blob/master/Datasets/Dataset9.md) from [VIROMOCK challenge](https://gitlab.com/ilvo/VIROMOCKchallenge). We put two fastq files are in the folder ~/Dataset9/raw (the "raw" subfolder is the defult fastq folder for PVseek). We use the same config file as the Dataset_8, and run the following command

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_pe.yaml -s /path/to/PVseek/Snakefile --config workDir=~/Dataset9 --core 16

After we check the result in the folder ~/Dataset9/report/report.txt, there are no exptected virus "Pistacia emaravirus".  Are somethings wrong?

**Note: The targeted virus sequences must be in the database for PVseek to map reads to them.**

Let's check whether "Pistacia emaravirus" is in the database

In [ ]:
!grep "Pistacia emaravirus" /path/to/PVseek/db/plantvirus_genome.fasta

No. The virus is not in the database. When we check [Virus-Host DB](https://www.genome.jp/virushostdb/) and [ICTV Virus Metadata Resource (VMR)](https://ictv.global/vmr), we can't find this virus. It means this virus is not collected by them.

Now we need to add Pistacia emaravirus reference genome sequences into our database. PVseek needs virus sequences and their annotations. It requires three files: 
  1. a fast format sequence file (ex. PiVB.fasta) 
  2. a tab delimited sequence taxonomy information file (ex. PiVB.gb_taxon.txt), which has three columns: sequence accessions, Taxonomy ID, and full lineage. 
  3. a tab delimited sequence information file (ex. PiVB.info.txt), which has three columns: sequence accesions, length, and description. 

We can use [NCBI Taxonomy Browser](https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi) to get the above information. We put the virus name "Pistacia emaravirus" in the search bar of Taxonomy Browser, then search, in the virus information page, we can see
```
    Taxonomy ID: 2507689
    Lineage( full )
        Viruses; Riboviria; Orthornavirae; Negarnaviricota; Polyploviricotina; Ellioviricetes; Bunyavirales; Fimoviridae; Emaravirus; unclassified Emaravirus
```

On the right side "Entrez records" table of the page, we click "Genome" direck link "1", we can see a table which contains all eight fragments (RNA 1, RNA 2 ... RNA 7) of Pistacia emaravirus and their sequence links. Through these eight links, we can get eight sequences, and their lengths and descriptions from NCBI. We can put these information in three files, which must be Unix (LF) format.

The three files, PiVB.fasta, PiVB.gb_taxon.txt, PiVB.info.txt, are in PVseek/test folder. 

After these three files are ready, we can merge them with our original database files using the following commands
```
cat PiVB.fasta >> /path/to/PVseek/db/plantvirus_genome.fasta
cat PiVB.gb_taxon.txt >> /path/to/PVseek/db/plantvirus.gb_taxon.txt
cat PiVB.info.txt >> /path/to/PVseek/db/plantvirus.info.txt
```

Since we update our sequences, before running PVseek, we need remove the previous index files by running
```
cd /path/to/PVseek/db
rm -rf *.ebwt
rm -rf *.bt2
rm -rf *.bwt
rm -rf *.mmi
```

After the new database is ready, we can rerun PVseek

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_pe.yaml -s /path/to/PVseek/Snakefile --config workDir=~/Dataset9 --core 16 -R pairRead_map
#-R pairRead_map asks snakemake to rerun the rule 'pairRead_map'

## <a id='single'></a> 2. Illumina single reads

We use the data [Dataset_10](https://gitlab.com/ilvo/VIROMOCKchallenge/-/blob/master/Datasets/Dataset10.md) from [VIROMOCK challenge](https://gitlab.com/ilvo/VIROMOCKchallenge) datasets from the European Plant Health Bioinformatics Network (PHBN).

### Step 1. set up config.yaml
The config file for singe reads is config_se.yaml in the PVseek folder. We can directly use it after changing database file paths.

Here are some explanation for config_se.yaml.

Our data file name is Dataset10_R1.fastq.gz. In the config_se.yaml file

    seq_type: 'se' #single read
    strand1: 'R1'  
    strand2: 'R2' #keep it, even no R2 read
    input_format: 'fastq.gz'

Fastp is used to trim reads. We use a sliding window (size 4) to trim reads with mean quality 20 and minimum length 30.  

    fastp_se_param: "--length_required 30 --cut_right --cut_window_size 4 --cut_mean_quality 20" 

Bwa is used to map reads to reference genomes. 

    mappingTool: bwa 
    bwa_param: "-B 4" #default bwa setting 

PVseek uses two thresholds to filter mapping results: minimum read mapped to a virus (readNumThd) and minimum genome coverage for a virus/fragment (genomeCovThd). You can setup them according to your positive controls and negative controls.

    readNumThd: 20 #minimu read number (20) for a virus to be selected
    genomeCovThd: 0.20 #minimum genome coverage (20%) for a virus to be selected

The virus coverage graph can be plotted if it's enabled by "yes".
    
    coverage: "yes"  #"no"   

The database file paths are needed to be changed to your local PVseek folder.

    virusDb: /path/to/PVseek/db/plantvirus_genome.fasta 
    virusTaxon: /path/to/PVseek/db/plantvirus.gb_taxon.txt 
    viralRefInfo: /path/to/PVseek/db/plantvirus.info.txt 

If you know some sequence annotations are questionalbe, please add their accesions in unwantedSeqId.txt

    badSeqIds: /paht/to/PVseek/db/unwantedSeqId.txt 

Please keep the other key/values in config_pe.yaml, do not change them.

### Step 2. run PVseek

Suppose Dataset_10 read file Dataset10_R1.fastq.gz is in a raw folder: ~/Dataset_10/raw, the work folder is ~/Dataset_10, the PVseek program is in /path/to/PVseek, 16 CPU cores are used. Then we can run a dry-test first

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_se.yaml -s /path/to/PVseek/Snakefile --config workDir=~/Dataset_10  --core 16 -n
#Note: if we put sequence files in the workDir/raw folder, we can ignore --fastqDir=/path/to/fastq

If dry-test is ok, we can run PVseek without "-n"

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_se.yaml -s /path/to/PVseek/Snakefile --config workDir=~/Dataset_10 --core 16

### Step 3. check results 

The filtered results are in the file ~/Dataset_10/report/report.txt.
The unfiltered results are in the file ~/Dataset_10/report/report.unfiltered.txt.
The read quality and numbers before and after trimming are in the file ~/Dataset_10/report/qcReadNumber.txt.

**Plum bark necrosis and stem pitting-associated virus (PBNSPaV) is detected.**

**Here is the report table** 

| Sample | TaxonId | ReadInTaxon | Reference | RefCoverage | Virus | TaxonPath | RefDescription |  
| :---: | :---: | :---: | :---: | :---: | :--- | :--- | :--- |  
|Dataset10|675077|51001|NC_009992.1|0.99|Plum bark necrosis stem pitting-associated virus|Viruses;Riboviria; Orthornavirae; Kitrinoviricota; Alsuviricetes; Martellivirales; Closteroviridae; Ampelovirus; Plum bark necrosis stem pitting-associated virus;|Plum bark necrosis and stem pitting-associated virus, complete genome|  

Note: The artificial new strain of Plum pox virus (PPV) is not detected. PVseek is not desigend for novel virus discovery. 

## <a id='nanopore'></a>3. Oxford Nanopore reads

We use the Oxford Nanopore's MinION data, NCBI accesion [SRR11431603](https://www.ncbi.nlm.nih.gov/sra/?term=SRR11431603), from [BioProject PRJNA612026 genereated by Michele Della Bartola et al. 2020](https://www.mdpi.com/1999-4915/12/4/478).

### Step 1. download data

We can download reads from NCBI using fastq-dump, which is a program from [NCBI SRA Toolkit](https://github.com/ncbi/sra-tools/wiki/01.-Downloading-SRA-Toolkit).

In [ ]:
mkdir -p ~/test_nanopore/raw
cd ~/test_nanopore/raw
fastq-dump --defline-seq '@$ac:$sl:$si:$sn:$si $ri:N:0:$sg' --defline-qual '+'  --gzip SRR11431603

#now we have a sequenc file SRR11431603.fastq.gz in the folder ~/test_nanopore/raw. The folder ~/test_nanopore is our work directory.

### Step 2. set up config.yaml
The config file for Nanopore reads is config_nanopore.yaml in the PVseek folder. We can directly use it after changing database file paths.

Here are some explanation for config_nanopore.yaml.

Our data file name is SRR11431603.fastq.gz. In the config_nanopore.yaml file

    seq_type: 'se' #single read
    strand1: '' #single read without R1  
    strand2: 'R2' #keep it, even no R2 read
    input_format: 'fastq.gz'

Fastp is used to trim reads. We filter reads with the minimum length 200 and the average quality greater than 9, hard clip first 10 bases.  

    fastp_nanopore_param: "--average_qual 9 --length_required 200 --trim_front1 10 --disable_quality_filtering" 

Minimap2 is used to map reads to reference genomes. 

    mappingTool: minimap2 
    minimap2_param: "-L --secondary=no -ax map-ont" #no secondary mapping 

PVseek uses two thresholds to filter mapping results: minimum read mapped to a virus (readNumThd) and minimum genome coverage for a virus/fragment (genomeCovThd). You can setup them according to your positive controls and negative controls.

    readNumThd: 10 #minimu read number (10) for a virus to be selected
    genomeCovThd: 0.30 #minimum genome coverage (30%) for a virus to be selected

The virus coverage graph can be plotted if it's enabled by "yes".
    
    coverage: "yes"  #"no"   

The database file paths are needed to be changed to your local the PVseek folder.

    virusDb: /path/to/PVseek/db/plantvirus_genome.fasta 
    virusTaxon: /path/to/PVseek/db/plantvirus.gb_taxon.txt 
    viralRefInfo: /path/to/PVseek/db/plantvirus.info.txt 

If you know some sequence annotations are questionalbe, please add their accesions in unwantedSeqId.txt

    badSeqIds: /paht/to/PVseek/db/unwantedSeqId.txt 

Please keep the other key/values in config_pe.yaml, do not change them.

### Step 3. run PVseek

Now our work folder is ~/test_nanopore and the read file is in ~/test_nanopore/raw. Suppose the PVseek program is in /path/to/PVseek, 16 CPU cores are used. Then we can run a dry-test first

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_nanopore.yaml -s /path/to/PVseek/Snakefile --config workDir=~/test_nanopore  --core 16 -n
#Note: if we put sequence files in the workDir/raw folder, we can ignore --fastqDir=/path/to/fastq

If dry-test is ok, we can run PVseek without "-n"

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_se.yaml -s /path/to/PVseek/Snakefile --config workDir=~/test_nanopore --core 16

### Step 4. check results 

The filtered results are in the file ~/test_nanopore/report/report.txt.
The unfiltered results are in the file ~/test_nanopore/report/report.unfiltered.txt.
The read quality and numbers before and after trimming are in the file ~/test_nanopore/report/qcReadNumber.txt.

**Potato virus S (PVS), Potato virus X (PVX), and Potato virus Y (PVY) are detected.**


## <a id='small'></a>4. Illumina small RNA reads

We use the Illumina small RNA data, NCBI accesion [SRR3152180](https://www.ncbi.nlm.nih.gov/sra?LinkName=biosample_sra&from_uid=4478417), from [BioProject PRJNA311127 genereated by Yi Zheng et al. 2017](https://www.sciencedirect.com/science/article/pii/S0042682216303166).

### Step 1. download data

We can download reads from NCBI using fastq-dump, which is a program from [NCBI SRA Toolkit](https://github.com/ncbi/sra-tools/wiki/01.-Downloading-SRA-Toolkit).

In [ ]:
mkdir -p ~/test_smallRNA/raw
cd ~/test_smallRNA/raw
fastq-dump --defline-seq '@$ac:$sl:$si:$sn:$si $ri:N:0:$sg' --defline-qual '+'  --gzip SRR3152180

#now we have a sequenc file SRR3152180.fastq.gz in the folder ~/test_smallRNA/raw. The folder ~/test_smallRNA is our work directory.

### Step 2. set up config.yaml
The config file for Illumina small RNA reads is config_sRNA.yaml in the PVseek folder. We can directly use it after changing database file paths.

Here are some explanation for config_sRNA.yaml.

Our data file name is SRR11431603.fastq.gz. In the config_nanopore.yaml file

    seq_type: 'se' #single read
    strand1: '' #single read without R1  
    strand2: 'R2' #keep it, even no R2 read
    input_format: 'fastq.gz'

Fastp is used to trim reads. We filter reads with a minimum length 15 and a maximum length 40, and trim polyX tail with a minimum length 5.  

    fastp_srna_param: "--length_required 15 --length_limit 40 --trim_poly_x --poly_x_min_len 5 --cut_right --cut_window_size 4 --cut_mean_quality 20"

If the adapter sequence is in the read (the read length >80bp), please add the adapter sequence in the above fastp_srna_param 
    
    "--adapter_sequence=CAGATCGGAAGAGCACACGT"

Bowtie is used to map small RNA reads to reference genomes. 

    mappingTool: bowtie
    bowtie_param: "-n 1 -k 1 -M 100 --best --strata" 

PVseek uses two thresholds to filter mapping results: minimum read mapped to a virus (readNumThd) and minimum genome coverage for a virus/fragment (genomeCovThd). You can setup them according to your positive controls and negative controls.

    readNumThd: 10 #minimu read number (10) for a virus to be selected
    genomeCovThd: 0.10 #minimum genome coverage (10%) for a virus to be selected

The virus coverage graph can be plotted if it's enabled by "yes".
    
    coverage: "yes"  #"no"   

The database file paths are needed to be changed to your local PVseek folder.

    virusDb: /path/to/PVseek/db/plantvirus_genome.fasta 
    virusTaxon: /path/to/PVseek/db/plantvirus.gb_taxon.txt 
    viralRefInfo: /path/to/PVseek/db/plantvirus.info.txt 

If you know some sequence annotations are questionalbe, please add their accesions in unwantedSeqId.txt

    badSeqIds: /paht/to/PVseek/db/unwantedSeqId.txt 

Please keep the other key/values in config_pe.yaml, do not change them.

### Step 3. run PVseek

Now our work folder is ~/test_smallRNA and the read file is in ~/test_smallRNA/raw. Suppose the PVseek program is in /path/to/PVseek, 16 CPU cores are used. Then we can run a dry-test first

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_sRNA.yaml -s /path/to/PVseek/Snakefile --config workDir=~/test_smallRNA  --core 16 -n
#Note: if we put sequence files in the workDir/raw folder, we can ignore --fastqDir=/path/to/fastq

If dry-test is ok, we can run PVseek without "-n"

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_sRNA.yaml -s /path/to/PVseek/Snakefile --config workDir=~/test_smallRNA --core 16

### Step 4. check results 

The filtered results are in the file ~/test_smallRNA/report/report.txt.
The unfiltered results are in the file ~/test_smallRNA/report/report.unfiltered.txt.
The read quality and numbers before and after trimming are in the file ~/test_smallRNA/report/qcReadNumber.txt.

**Potato virus S (PVS), Potato virus X (PVX), and Potato virus T (PVT) are detected.**
Note: Tobacco vein clearing virus with 66 reads and 0.14 genome coverage is in the report. This is a false positive. The reads can also be mapped to the plant host. If you use 20% (0.2) as the genome coverage threhold, it's filtered.

## <a id='hiplex'></a>5. HiPlex/Amplicon reads

We use the Illumina HiPlex data, NCBI accesion [SRR22107999](https://www.ncbi.nlm.nih.gov/sra/SRX18088110[accn]), from [BioProject PRJNA896155 genereated by Larissa Carvalho Costa et al. 2022](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9791224/).

### Step 1. download data

We can download reads from NCBI using fastq-dump, which is a program from [NCBI SRA Toolkit](https://github.com/ncbi/sra-tools/wiki/01.-Downloading-SRA-Toolkit).

In [ ]:
mkdir -p ~/test_hiplex/raw
cd ~/test_hiplex/raw
fastq-dump --defline-seq '@$ac:$sl:$si:$sn:$si $ri:N:0:$sg' --defline-qual '+'  --gzip SRR22107999

#now we have a sequenc file SRR22107999.fastq.gz in the folder ~/test_hiplex/raw. The folder ~/test_hiplex is our work directory.

### Step 2. set up config.yaml
The config file for HiPlex/Amplicon reads is config_hiplex.yaml in the PVseek folder. We can directly use it after changing database file paths.

Here are some explanation for config_hiplex.yaml.

Our data file name is SRR22107999.fastq.gz. In the config_nanopore.yaml file

    seq_type: 'se' #single read
    strand1: '' #single read without R1  
    strand2: 'R2' #keep it, even no R2 read
    input_format: 'fastq.gz'

Fastp is used to trim reads. We use a sliding window (size 4) to trim reads with mean quality 20 and minimum length 100.  

    fastp_hiplex_param: "--length_required 100 --cut_right --cut_window_size 4 --cut_mean_quality 20"  

BWA is used to map reads to reference genomes. 

    mappingTool: bwa 
    bwa_param: " -B 4 "

PVseek uses two thresholds to filter mapping results: minimum read mapped to a virus (readNumThd) and minimum genome coverage for a virus/fragment (genomeCovThd). You can setup them according to your positive controls and negative controls.

    readNumThd: 1000 #minimu read number (1000) for a virus to be selected
    genomeCovThd: 0 # 0 means no genome coverage filter for amplicons

The virus coverage graph can be plotted if it's enabled by "yes".
    
    coverage: "no" #"yes"   

The database file paths are needed to be changed to your local database folder. Note: This database is specific for targeted viruses, not a general plant virus database. PVseek has an example for Pomes HiPlex.

    virusDb: /path/to/PVseek/db/Pomes_HiPlex_refSeq.fasta 
    virusTaxon: /path/to/PVseek/db/Pomes_HiPlex_refSeq.gb_taxon.txt 
    viralRefInfo: /path/to/PVseek/db/Pomes_HiPlex_refSeq.info.txt

If you know some sequence annotations are questionalbe, please add their accesions in unwantedSeqId.txt

    badSeqIds: /paht/to/PVseek/db/unwantedSeqId.txt 

Please keep the other key/values in config_pe.yaml, do not change them.

### Step 3. run PVseek

Now our work folder is ~/test_hiplex and the read file is in ~/test_hiplex/raw. Suppose the PVseek program is in /path/to/PVseek, 16 CPU cores are used. Then we can run a dry-test first

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_hiplex.yaml -s /path/to/PVseek/Snakefile --config workDir=~/test_hiplex  --core 16 -n
#Note: if we put sequence files in the workDir/raw folder, we can ignore --fastqDir=/path/to/fastq

If dry-test is ok, we can run PVseek without "-n"

In [ ]:
!snakemake  --configfile /path/to/PVseek/config_hiplex.yaml -s /path/to/PVseek/Snakefile --config workDir=~/test_hiplex --core 16

### Step 4. check results 

The filtered results are in the file ~/test_hiplex/report/report.txt.
The unfiltered results are in the file ~/test_hiplex/report/report.unfiltered.txt.
The read quality and numbers before and after trimming are in the file ~/test_hiplex/report/qcReadNumber.txt.

**Apple chlorotic leaf spot virus(ACLSV),  Apple green crinkle associated virus(AGCaV), Apple hammerhead viroid(AHVd), ASGV(Apple stem grooving virus), Apple stem pitting virus(ASPV), Citrus concave gum-associated virus(CCGaV) are detected.**
